# Homework 3 Beginnings:
Starting with the notes from class, lecture 5: Black Hole Imaging, we have a set of ray equations for a schwarzschild metric.

### Initialization factors:

spin: $0 < a_0 < 1$

inclination: $0 < \theta_0 < \pi$

angular momentum: $L$


### Notation:

The Sky Plane, $S$ with cartesian coordinates $(X,Y)$

Y: Parallel to the projection of hole spin on the sky

Momentum vector: $k = \frac{dx}{d\xi}$ with affine parameter $\xi = 0$ at $S$ and $\xi < 0$ near the BH.

Introduce coordine $s = \frac{1}{r}$ and $\mu = cos\theta$

A parameter of the hole is related to the angular momentum by $a = cJ/GM^2$, we will measure this length in terms of mass to make it unitless.

### Equations needing Evaluation
For now we are interested in stationary sources, so we have dropped the $t$ parameter.

**Energy and angular momentum are conserved along the ray:*

$\mathcal{E} = -k_t = 1, k^t = \frac{dt}{d\xi} = 1$ at $S$ where $1 << r << D$

$L = r^2 sin^2\theta \frac{d\phi}{d\xi} = r^2 sin^2\theta_0 \frac{d\phi}{dt} = -X sin\theta_0$

**The Carter constant:**

$Q = k_{\theta}^2 + L^2 cot(\theta)-a^2cos^2\theta$

Note, at S, $k_{\theta}= Y$.

### ODEs to solve:
If needing $t$:

$t' = \frac{dt}{d\lambda} = -\frac{\Sigma^2}{\Delta}(1-\omega L)$

$t' = -\frac{1+as^2(a+2as-2Ls+a(1+s(-2+a^2s))\mu^2}{s^2(1+s(-2+a^2s))}$

$\phi$ **Equation:**

$\phi' = \frac{d\phi}{d\lambda}$

$\phi' = \frac{L+2as-2Ls+as(-2+aLs)\mu^2}{(1-s(-2+a^2s))(-1+\mu^2)}$

$\mu$ **Equation:**

$\mu'' = -\mu(L^2+Q+a^2(-1+2\mu^2))$

$s$ **Equation:**

$s'' = s(-6aLs+(-1+3s)(L^2+Q)+a^2(1+s(3-2sQ)))$

In [11]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import ode #ode solver

In [24]:
class bh_vis:
    def __init__(self,size,a0=0.5,inc=np.pi/2):
        #initialize class
        #size should be odd so we can have a centered origin
        self.XY = np.zeros(size) #image grid
        self.x = np.linspace(-int((size[1]-1)/2),int((size[1]-1)/2),num=size[1])
        self.y = np.linspace(-int((size[0]-1)/2),int((size[0]-1)/2),num=size[0])
        self.size = size
        self.a0 = a0 #spin/mass parameter
        self.inc = inc
        self.L = -self.x*np.sin(inc) #an array of momentum based on X coordinate
        
        #may need to check math on this
        self.Q = self.y**2 + self.L**2*np.cos(inc)/np.sin(inc)-a0*np.cos(inc)**2 #Carter constant conserved
        print('Instance initialized:\nsize: {0}\nspin: {1}\ninclination: {2}'.format(size,a0,inc))
        return

    def twoDtrace(self, **kwargs):
        #perform a 2D trace as seen in cell 182 of the lecture notes
        return 0
    
    def solve_mu(self,initialconds):
        #solves the second order DE given the position and 1st derivative ICs
        #independent of r
        mu = 0
        return mu
    
    def solve_s(self,initialconds):
        s = 0
        return s
    
    def threeDtrace(self,**kwargs):
        #if we get around to it, plot real image which requires a three dimensional trace
        return 0
    
    def source_init(self,params):
        #set up the source, likely an equatorial disk of uniform brightness
        return 0
    
        

In [25]:
#example call
test1 = bh_vis((33,33),a0=0.9,inc=np.pi/3)

print('X Coordinates:')
print(test1.x)

print('Q constant:')
print(test1.Q)

Instance initialized:
size: (33, 33)
spin: 0.9
inclination: 1.0471975511965976
X Coordinates:
[-16. -15. -14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.
  -2.  -1.   0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.
  12.  13.  14.  15.  16.]
Q constant:
[ 3.66626252e+02  3.22202858e+02  2.80645490e+02  2.41954147e+02
  2.06128829e+02  1.73169537e+02  1.43076270e+02  1.15849029e+02
  9.14878129e+01  6.99926224e+01  5.13634573e+01  3.56003175e+01
  2.27032032e+01  1.26721143e+01  5.50705081e+00  1.20801270e+00
 -2.25000000e-01  1.20801270e+00  5.50705081e+00  1.26721143e+01
  2.27032032e+01  3.56003175e+01  5.13634573e+01  6.99926224e+01
  9.14878129e+01  1.15849029e+02  1.43076270e+02  1.73169537e+02
  2.06128829e+02  2.41954147e+02  2.80645490e+02  3.22202858e+02
  3.66626252e+02]
